In [41]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from  tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

input_path = 'path'
output_path = 'path'


print("Done!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Done!


**After we have imported all the necessary libraries and specified the path for the data, we will declare several variables for working with photos.**

In [42]:
IMG_SIZE = (160, 160)
BATCH_SIZE = 32
epochs=10
tuning_epochs=8
learning_rate=0.0001
validation_koef = 5

print("Done!")

Done!


**Now let's compose our train and validation dataset.**

In [43]:
train_dataset = image_dataset_from_directory(input_path,
                                             shuffle=True,
                                             subset='training',
                                             seed=1,
                                             validation_split=0.2,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

validation_dataset = image_dataset_from_directory(input_path,
                                             shuffle=True,
                                             subset='validation',
                                             seed=1,
                                             validation_split=0.2,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 7553 files belonging to 3 classes.
Using 6043 files for training.
Found 7553 files belonging to 3 classes.
Using 1510 files for validation.


**There are a total of 32 batches of batch data in the verification set (one batch contains 32 images), and the 32 batches are divided into 26 and 6, 2 parts (verification and testing)**

**So now let's take a data, what we need and create test dataset.**

In [44]:
validation_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(validation_batches//validation_koef)
validation_dataset = validation_dataset.skip(validation_batches//validation_koef)

print("Done!")

Done!


**Now let's configure the data for us.**

In [45]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

print("Done!")

Done!


**Let's increase our data and change it a little. Very little))**

**First of make flip horizontally and vertically, spin, zoom, add contrast of each channel of each picture ,shifted up and down by 20%, left and right by 20%.**

In [46]:
data_change=tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(.5, .2),
    tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.1),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(
        height_factor=0.1, width_factor=0.1)
])

print("Done!")

Done!


**Let's making the base model.**

In [47]:
base_model=tf.keras.applications.EfficientNetB0(input_shape=IMG_SIZE+(3,),
                                    include_top=False,
                                    weights='imagenet',
                                    drop_connect_rate=0.4
                                    )
base_model.trainable=False
model=tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(1))

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 5, 5, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1281      
Total params: 4,050,852
Trainable params: 1,281
Non-trainable params: 4,049,571
_________________________________________________________________
None


**Good, all works. Now let's make some modification with layer(normalize, pooling, dense and drop some data).**

In [48]:
input=tf.keras.Input(IMG_SIZE+(3,))
x=base_model(data_change(input),
             training=False)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dropout(0.2)(x)
x=tf.keras.layers.BatchNormalization()(x)
output=tf.keras.layers.Dense(1)(x)
model=tf.keras.Model(input,output)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 160, 160, 3)       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 5, 5, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1280)              5120      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1281

**Now let's get down to optimization. We will optimize by Adam's method.**

In [49]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

print("Done!")

Done!


**And fit our model.**

In [17]:
history=model.fit(train_dataset,
          epochs=epochs,
          validation_data=validation_dataset
         )

Epoch 1/10
189/189 [==============================] - 1906s 10s/step - loss: 0.3614 - accuracy: 0.8288 - val_loss: 0.0685 - val_accuracy: 0.9746
Epoch 2/10
189/189 [==============================] - 266s 1s/step - loss: 0.1639 - accuracy: 0.9347 - val_loss: 0.0417 - val_accuracy: 0.9828
Epoch 3/10
189/189 [==============================] - 266s 1s/step - loss: 0.1543 - accuracy: 0.9384 - val_loss: 0.0342 - val_accuracy: 0.9861
Epoch 4/10
189/189 [==============================] - 263s 1s/step - loss: 0.1305 - accuracy: 0.9504 - val_loss: 0.0361 - val_accuracy: 0.9836
Epoch 5/10
189/189 [==============================] - 262s 1s/step - loss: 0.1145 - accuracy: 0.9580 - val_loss: 0.0332 - val_accuracy: 0.9820
Epoch 6/10
189/189 [==============================] - 266s 1s/step - loss: 0.1133 - accuracy: 0.9522 - val_loss: 0.0325 - val_accuracy: 0.9861
Epoch 7/10
189/189 [==============================] - 262s 1s/step - loss: 0.1184 - accuracy: 0.9533 - val_loss: 0.0310 - val_accuracy: 0.98

**Let's save our model.**

In [53]:
wqe=model.evaluate(test_dataset)
print(wqe)
model.save('/content/drive/MyDrive')
print("Done!")

9/9 [==============================] - 13s 1s/step - loss: 0.6880 - accuracy: 0.0056
[0.7190898656845093, 0.0069444444961845875]
INFO:tensorflow:Assets written to: /content/drive/MyDrive/assets
Done!
